# Human-Robot Collaboration - Pose Forecasting Training

Questo notebook serve per avviare il training su Google Colab copiando i dati in locale per massima velocità e stabilità.

### Istruzioni:
1. Carica l'intera cartella `human-robot-collaboration` sul tuo Google Drive.
2. Esegui le celle in sequenza.

In [ ]:
# 1. Collega Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
# 2. Copia il progetto nella RAM locale di Colab
# Questo passaggio è FONDAMENTALE per evitare errori di disconnessione e lentezza
import shutil
import os
import time
from tqdm.notebook import tqdm

# Percorso sorgente (Drive)
source_dir = '/content/drive/My Drive/human-robot-collaboration'
# Percorso destinazione (Locale Colab)
dest_dir = '/content/human-robot-collaboration'

print(f"Inizio copia da {source_dir} a {dest_dir}...")

if os.path.exists(dest_dir):
    print("Rimozione vecchia cartella locale...")
    shutil.rmtree(dest_dir)

# Funzione per copiare con progress bar
def copy_with_progress(src, dst):
    # Conta i file totali
    print("Conteggio file in corso...")
    total_files = 0
    for root, dirs, files in os.walk(src):
        total_files += len(files)
    
    print(f"Trovati {total_files} file da copiare.")
    
    # Copia
    with tqdm(total=total_files, unit='file', desc='Copia file') as pbar:
        for root, dirs, files in os.walk(src):
            # Calcola path relativo
            rel_path = os.path.relpath(root, src)
            dest_root = os.path.join(dst, rel_path)
            
            # Crea cartelle
            os.makedirs(dest_root, exist_ok=True)
            
            for file in files:
                # Ignora file di Google Drive (gdoc, gsheet, ecc.) che non si possono copiare
                if file.endswith(('.gdoc', '.gsheet', '.gslides')):
                    pbar.update(1)
                    continue

                src_file = os.path.join(root, file)
                dst_file = os.path.join(dest_root, file)
                shutil.copy2(src_file, dst_file)
                pbar.update(1)

start_time = time.time()
copy_with_progress(source_dir, dest_dir)
end_time = time.time()

print(f"Copia completata in {end_time - start_time:.2f} secondi!")

# Spostiamoci nella cartella locale
os.chdir(dest_dir)
print(f"Working directory impostata su: {os.getcwd()}")

In [ ]:
# 3. Installa le dipendenze
!pip install -r requirements.txt

In [ ]:
# 4. Avvia il Training
# Ora sta leggendo dal disco locale di Colab, quindi sarà molto veloce

!python train_predictor.py --epochs 50 --batch_size 32

In [ ]:
# 5. Salva i risultati su Drive (Checkpoints e Logs)
# Esegui questa cella alla fine del training per salvare tutto su Drive
import shutil
import os
from datetime import datetime

# Cartelle locali (dove ha scritto il training)
local_checkpoints = '/content/human-robot-collaboration/checkpoints'
local_runs = '/content/human-robot-collaboration/runs'

# Cartelle di destinazione su Drive
drive_base = '/content/drive/My Drive/human-robot-collaboration'
drive_checkpoints = os.path.join(drive_base, 'checkpoints_backup')
drive_runs = os.path.join(drive_base, 'runs_backup')

print("Inizio backup su Google Drive...")

# 1. Backup Checkpoints
if os.path.exists(local_checkpoints):
    print(f"Copia checkpoints da {local_checkpoints} a {drive_checkpoints}...")
    # Copia ricorsiva che sovrascrive/aggiorna
    if os.path.exists(drive_checkpoints):
        shutil.rmtree(drive_checkpoints)
    shutil.copytree(local_checkpoints, drive_checkpoints)
    print("Checkpoints salvati!")
else:
    print("Nessuna cartella checkpoints trovata in locale.")

# 2. Backup Runs (TensorBoard logs)
if os.path.exists(local_runs):
    print(f"Copia logs da {local_runs} a {drive_runs}...")
    if os.path.exists(drive_runs):
        shutil.rmtree(drive_runs)
    shutil.copytree(local_runs, drive_runs)
    print("Logs salvati!")
else:
    print("Nessuna cartella runs trovata in locale.")

print(f"Backup completato alle {datetime.now().strftime('%H:%M:%S')}!")

In [ ]:
# 6. Avvia TensorBoard
%load_ext tensorboard
%tensorboard --logdir runs/